In [6]:
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

In [7]:
# setup
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '#########'
VIEW_ID = '136386382' # coloque aqui o TableID do Analytics

In [8]:
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics
 
def get_report(analytics):
    return analytics.reports().batchGet(body={'reportRequests': [{
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': 'today', 'endDate': 'today'}],
        'dimensions': [
            {'name': 'ga:dateHourMinute'},
            {'name': 'ga:pagePath'},
            {'name': 'ga:pageTitle'},
            {'name': 'ga:previousPagePath'},
            {'name': 'ga:city'},
            {'name': 'ga:browser'},
            {'name': 'ga:browserVersion'},
            {'name': 'ga:operatingSystem'},
            {'name': 'ga:screenResolution'}
        ], 'metrics': [
            {'expression': 'ga:pageviews'},
            {'expression': 'ga:timeOnPage'},
            {'expression': 'ga:domContentLoadedTime'},
            {'expression': 'ga:sessions'}
        ]
    }]}).execute()
 
def response2df(response):
    report = response.get('reports', [])[0] # expected just one report
    # headers
    header_dimensions = report.get('columnHeader', {}).get('dimensions', [])
    header_metrics = [value['name'] for value in report.get('columnHeader', {}).get('metricHeader', {}).get('metricHeaderEntries', [])]
    headers = header_dimensions + header_metrics
    headers = list(map((lambda x: x.split(':', 1)[-1]), headers)) # removes "ga:" from each column
    # values
    values = []
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        values_dimensions = row.get('dimensions', [])
        values_metrics = row.get('metrics', [])[0].get('values', [])
        values.append(values_dimensions + values_metrics)
    # to dataframe
    df = pd.DataFrame(columns=headers, data=values)
    return df

In [9]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)
df = response2df(response)

In [10]:
display(df)
display(df.shape)

,dateHourMinute,pagePath,pageTitle,previousPagePath,city,browser,browserVersion,operatingSystem,screenResolution,pageviews,timeOnPage,domContentLoadedTime,sessions
0,202006160000,/,"Olé Consignado - Empréstimo Consignado, Cartão...",(entrance),Manaus,Chrome,83.0.4103.101,Android,360x640,1,24.0,0,1
1,202006160000,/,Olé Consignado | Minhas solicitações,/cliente/,Uberlandia,Samsung Internet,11.2,Android,360x780,1,0.0,0,0
2,202006160000,/cartao,Olé Consignado | Cartão de crédito,/cliente/cartao,Campina Grande,Chrome,83.0.4103.106,Windows,1600x900,1,24.0,0,0
3,202006160000,/cartao,Olé Consignado | Cartão de crédito,/cliente/cartao,Uberlandia,Safari,13.0.5,iOS,375x812,1,21.0,0,0
4,202006160000,/cartao/faturas,Olé Consignado | Cartão de crédito,/cliente/cartao/faturas,Uberlandia,Safari,13.0.5,iOS,375x812,1,20.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,202006160015,/cliente/solicitacao/descontos,Olé Consignado | Simulação,/solicitacao/data-nascimento,Brasilia,Chrome,83.0.4103.101,Android,363x745,1,0.0,0,0
996,202006160015,/cliente/solicitacao/estado-fonte-pagadora,Olé Consignado | Simulação,/cliente/solicitacao,Franco da Rocha,Chrome,83.0.4103.97,Windows,1536x864,1,0.0,0,0
997,202006160015,/cliente/solicitacao/estado-fonte-pagadora,Olé Consignado | Simulação,/solicitacao,Lagarto,Chrome,79.0.3945.116,Android,412x846,1,0.0,0,0
998,202006160015,/cliente/solicitacao/estado-fonte-pagadora,Olé Consignado | Simulação,/solicitacao/estado-fonte-pagadora,Franco da Rocha,Chrome,83.0.4103.97,Windows,1536x864,1,0.0,0,0


(1000, 13)